# Objective Ranking Signals Quickstart

Add new factors to the ranking of your search results with Ranking Signals.

Read the full [Quickstart Doc](TODO).

## Setup

Begin by obtaining an API key and installing the SDK:

To get an API key, [join the waitlist](https://objective.inc/contact).

```bash
pip install objective-sdk
```

In [1]:
!pip install requests objective-sdk

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached objective_sdk-4.0.9-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached certifi-2024.6.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached more_itertools-10.3.0-py3-none-any.whl.metadata (36 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached objective_sdk-4.0.9-py3-none-any.whl (11 kB)
Using cached certifi-2024.6.2-py3-none-any.whl (164 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl (119 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached more_itertools-10.3.0-py3-none-any.whl (59 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 3.4 MB/s eta 0:00:00


## Adding Data

Begin by adding Objects to the Object Store. The code below will download 10,000 example e-commerce products and upsert them to the Object Store. In this example we are building an e-commerce search platform:

In [28]:
import json
import requests
from objective import Client

client = Client(api_key="sk_MV7WU6l3Tenp")

data = json.loads(requests.get("https://d11p8vtjlacpl4.cloudfront.net/demos/ecommerce/hm-10k.json").text)

client.object_store.upsert_objects([{"id": str(obj.get("article_id")), "object": obj} for obj in data])

Successfully upserted 10000 objects.


BatchOperation(success=10000, failures=0)

## Building an Index

Build a `multimodal` index out of the Objects pushed to the API with `searchable` and `filterable` fields:

In [29]:
index = client.indexes.create_index(
    index_type="multimodal", # Specify the type of index as 'text'
    fields={
      # Defines fields that will be indexed, their values will influence relevance
      "searchable": ["prod_name", "department_name", "detail_desc", "garment_group_name", "index_group_name", "index_name", "perceived_colour_master_name", "perceived_colour_value_name"],
      # Defines fields that will be filterable
      "filterable": ["department_name", "colour_group_name", "garment_group_name", "index_group_name", "index_name"]
    }
)
index.status(watch=True)

Creating index...
Index idx_Zcfneus0euFo successfully created.
Index idx_Zcfneus0euFo not initialized yet, sleeping for 30 seconds.


Progress: [###################################################################################################] 100%


{'status': {'READY': 10000, 'ERROR': 3, 'PROCESSING': 0, 'UPLOADED': 0}}

## Run a query

Next, run a query and let's look at the results. But before running the query we'll also create a function (`display_results`) to help us display the results.

In [23]:

def display_results(results):
    for obj in results['results']:
        from IPython.display import Image, display, HTML
        data = obj["object"]
        image_html = f'<div style="display: flex; align-items: center;">' \
                     f'<div style="flex: 1;"><img src="{data["image_url"]}" height="120"></div>' \
                     f'<div style="flex: 2;">' \
                     f'<p>ID: {obj["id"]}</p>' \
                     f'<p>Product Name: {data["prod_name"]}</p>' \
                     f'<p>{data["index_name"]} &raquo; {data["product_group_name"]} &raquo; {data["product_type_name"]}</p>' \
                     f'<p>Color: {data["colour_group_name"]} - {data["colour_group_code"]}</p>'\
                     f'</div></div>'
        
        display(HTML(image_html))

results = index.search("t-shirt", object_fields="*", limit=20)
display_results(results)

In [27]:

ranking_expr = "uncalibrated_relevance * if(object.color_code == 72, 1.05, 1)"

results = index.search("t-shirt", object_fields="*", limit=20, ranking_expr=ranking_expr)
display_results(results)

TypeError: Index.search() got an unexpected keyword argument 'ranking_expr'